In [3]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.6/851.6 kB 4.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')
import os
os.environ['PATH'] = '/Users/ravi.tej/anaconda3/envs/bertopicenv/bin:/Users/ravi.tej/anaconda3/condabin:/usr/bin:/bin:/usr/sbin:/sbin'

In [2]:
import anthropic

In [3]:
client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get('ANTHROPIC_KEY'),
)

In [47]:
portfolio_system_prompt = '''You are an expert personal financial manager. You answer user queries related to their portfolio succinctly using data of the portfolio that you have extracted. If the extracted data has any additional relevant insights, share top insights from there as well.  Remember that you have extracted the data yourself. If the data does not answer the query then apologize and share what you were able to find instead. Be short, direct and avoid repetition. Use a maximum of 1 decimal place everywhere'''

In [58]:
u_query = 'what is the XIRR of my portfolio'

In [43]:
k = [{'text_box': {'data': [{'Total Profit': 90228.63013599999}],
   'value_type': 'rupees',
   'chart_title': 'Total Profit'}},
 {'bar_chart': {'value_type': 'rupees',
   'chart_title': 'Top Holdings by Total Profit',
   'data': [{'BSE Ltd': 24440.699999999997},
    {'Multi Commodity Exchange of India Ltd': 18851.199999999997},
    {'Tata Motors Ltd': 10460.25},
    {'Tata Power Company Ltd': 6023.7},
    {'Oil and Natural Gas Corporation Ltd': 5898.75},
    {'Central Depository Services (India) Ltd': 5788.4},
    {'Colgate-Palmolive (India) Ltd': 4411.400000000001},
    {'Titan Company Ltd': 4379.700000000001},
    {'Indian Railway Catering and Tourism Corporation Ltd': 3577.2000000000007},
    {'others': 6397.330136000002}]}}]

In [13]:
import json

In [24]:
k[0].values()[0]

TypeError: 'dict_values' object is not subscriptable

In [31]:
# d = json.dumps([list(x.values())[0]['data'] for x in k])
d = json.dumps(k)

In [15]:
json.dumps([x['data'] for key, x in k.items()])

AttributeError: 'list' object has no attribute 'items'

In [59]:
message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=128,
    messages=[
        {"role": "user", "content": f'{portfolio_system_prompt}. user_query: {u_query}. data: {d}'}
    ]
)
print(message.content)

[ContentBlock(text='Based on the data provided, the XIRR (Extended Internal Rate of Return) of your portfolio is not directly available. The data shows the total profit of your portfolio as ₹90,228.63, but does not include information about the investment timeline or cash flows required to calculate the XIRR.\n\nWithout the additional data needed to calculate the XIRR, I am unable to provide a precise figure for the XIRR of your portfolio. However, the data does provide insights into the top holdings in your portfolio by total profit, which can be useful information for understanding the performance of your investments', type='text')]


In [83]:
message.content[0].text

'Based on the data provided, the XIRR (Extended Internal Rate of Return) of your portfolio is not directly available. The data shows the total profit of your portfolio as ₹90,228.63, but does not include information about the investment timeline or cash flows required to calculate the XIRR.\n\nWithout the additional data needed to calculate the XIRR, I am unable to provide a precise figure for the XIRR of your portfolio. However, the data does provide insights into the top holdings in your portfolio by total profit, which can be useful information for understanding the performance of your investments'

In [60]:
import json

In [61]:
json.loads('')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [55]:
message

Message(id='msg_01DGq3YnrWdCoNjHsMiDzHFs', content=[ContentBlock(text='Based on the extracted data, here are the shares that have beaten the index (assuming a benchmark index return is implied):\n\n1. BSE Ltd\n2. Multi Commodity Exchange of India Ltd\n3. Tata Motors Ltd\n4. Tata Power Company Ltd\n5. Oil and Natural Gas Corporation Ltd\n6. Central Depository Services (India) Ltd\n7. Colgate-Palmolive (India) Ltd\n8. Titan Company Ltd\n9. Indian Railway Catering and Tourism Corporation Ltd\n\nThe top insight is that the portfolio has generated a total profit of Rs.', type='text')], model='claude-3-sonnet-20240229', role='assistant', stop_reason='max_tokens', stop_sequence=None, type='message', usage=Usage(input_tokens=359, output_tokens=128))

In [6]:
def cal_price(inp, out):
    in_price = 0.25 * inp/1000000
    out_price = 1.25 * out/1000000
    return in_price + out_price

In [57]:
cal_price(359, 128) * 90 * 12

0.26973

In [9]:
5 * 80

400

In [62]:
import requests

In [70]:
q = 'what is my most profitable stock'

In [66]:
r = requests.post('http://127.0.0.1:8000/search_query_portfolio', json = {'query_text': 'what is my most profitable stock', 'userId': 1})

In [68]:
import json

In [73]:
s = json.dumps(json.loads(r.text)['graphData'])

In [74]:
s

'[{"bar_chart": {"chart_title": "Top Holdings by Total Profit", "data": [{"BSE Ltd": 23761.199999999997}, {"Multi Commodity Exchange of India Ltd": 19424.199999999997}, {"Tata Motors Ltd": 10758.25}, {"Central Depository Services (India) Ltd": 6000.9}, {"Tata Power Company Ltd": 5971.499999999999}, {"Oil and Natural Gas Corporation Ltd": 5914.35}, {"Titan Company Ltd": 4696.299999999999}, {"Colgate-Palmolive (India) Ltd": 4527.000000000001}, {"Indian Railway Catering and Tourism Corporation Ltd": 3486.4500000000007}, {"others": 7209.4201360000025}], "value_type": "rupees"}}]'

In [78]:
r1 = requests.get('http://127.0.0.1:8000/summarize_search_v2', params = {'query_text': q, 'userId': 1, 'portfolio_data': s})

In [79]:
r1.text

'Based on the extracted portfolio data, the most profitable stock is BSE Ltd, with a total profit of ₹23,761.2.'